## Function Definitions

In [3]:
import json
import uuid
from functools import partial


def add(args: str):
    args = json.loads(args)
    return str(float(args["a"]) + float(args["b"]))


def sub(args: str):
    args = json.loads(args)
    return str(float(args["a"]) - float(args["b"]))


def mult(args: str):
    args = json.loads(args)
    return str(float(args["a"]) * float(args["b"]))


def div(args: str):
    args = json.loads(args)
    return str(float(args["a"]) / float(args["b"]))


def get_oai_response(model, functions, msgs, api_key, base_url):
  import openai
  openai.api_key = api_key ## Add your API key here
  openai.base_url = base_url
  print(f"Pointing to URL: {base_url}")
  
  try:
    completion = openai.chat.completions.create(
      model=model,
      temperature=0.1,
      messages=msgs,
      tools=functions,
      tool_choice="auto",
      # functions=functions,
      # function_call="auto",
      stream=False,
    )
    return completion.choices[0]
  except Exception as e:
    print(e)


def insert_tool_response(res, msgs):
    # for tool_call in res.message.tool_calls:
    #     print(f"Tool Call: {tool_call.id}, {tool_call.function}")
    assistant_message = res.message
    tool_calls = []
    for tool_call in assistant_message.tool_calls:
        tool_calls.append( {
                            "id": tool_call.id,
                            "function": {"name": tool_call.function.name,
                                        "arguments": tool_call.function.arguments},
                            "type": "function",
                        })
    msgs.append({"role": "assistant",  "tool_calls": tool_calls})
    
    for i, tool_call in enumerate(assistant_message.tool_calls):
        if tool_call.function.name == "getCurrentWeather":
            print()
            l = len((json.loads(assistant_message.tool_calls[i].function.arguments))["location"])
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"temprature is {(i+1) * 50 + l } degree"})
        elif tool_call.function.name == "calculate_distance":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"Distance is {(i+1) * 50} miles."})
        elif tool_call.function.name == "generate_password":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"Password generated: {uuid.uuid4().hex[:8]}"})
        elif tool_call.function.name == "orderUmbrella":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"Order placed. the price is {(i+1) * 10} dollars."})
        elif tool_call.function.name == "addition":
            msgs.append({
                "role": "tool",
                "name": "addition",
                "content": add(tool_call.function.arguments),
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "subtraction":
            msgs.append({
                "role": "tool",
                "name": "subtraction",
                "content": sub(tool_call.function.arguments),
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "multiplication":
            msgs.append({
                "role": "tool",
                "name": "multiplication",
                "content": mult(tool_call.function.arguments),
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "division":
            msgs.append({
                "role": "tool",
                "name": "division",
                "content": div(tool_call.function.arguments),
                "tool_call_id": tool_call.id
            })
    
    return msgs

def run_completion(chat_method, user_query, msgs=[]):
    system_prompt = "You are a helpful assistant."
    functions = [
        # {"type": "function","function":{"name":"calculate_distance","description":"Calculate the distance between two locations","parameters":{"type":"object","properties":{"origin":{"type":"string","description":"The starting location"},"destination":{"type":"string","description":"The destination location"},"mode":{"type":"string","description":"The mode of transportation"}},"required":["origin","destination","mode"]}}},{"type": "function","function":{"name":"generate_password","description":"Generate a random password","parameters":{"type":"object","properties":{"length":{"type":"integer","description":"The length of the password"}},"required":["length"]}}},
        {
            'type': 'function',
            'function': {
                'name': 'addition',
                'description': "Adds two numbers together",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to add',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Second number to add',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'subtraction',
                'description': "Subtracts two numbers",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to be subtracted from',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Number to subtract',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'multiplication',
                'description': "Multiply two numbers together",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to multiply',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Second number to multiply',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'division',
                'description': "Divide two numbers",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to use as the dividend',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Second number to use as the divisor',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
      {
          "type": "function",
        "function": {
          "name": "getCurrentWeather",
          "description": "Get the weather in location",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
              "unit": {"type": "string", "enum": ["c", "f"]}
            },
            "required": ["location"]
          }
        }
      },
      {    "type": "function",
        "function":
        {
            "name": "orderUmbrella",
            "description": "Do this to help user to order an umbrella online", 
            "parameters": {
                "type": "object",
                "properties": {
                    "number_to_buy": {
                        "type": "integer",
                        "description": "the amount of umbrellas to buy"
                    }
                },
                "required": [
                    "number_to_buy"
                ]
            }
        }},
      {"type": "function","function":{"name":"calculate_distance","description":"Calculate the distance between two locations","parameters":{"type":"object","properties":{"origin":{"type":"string","description":"The starting location"},"destination":{"type":"string","description":"The destination location"},"mode":{"type":"string","description":"The mode of transportation"}},"required":["origin","destination","mode"]}}},{"type": "function","function":{"name":"generate_password","description":"Generate a random password","parameters":{"type":"object","properties":{"length":{"type":"integer","description":"The length of the password"}},"required":["length"]}}}
       ]
    # functions = [{"type": "function","function":{"name":"calculate_distance","description":"Calculate the distance between two locations","parameters":{"type":"object","properties":{"origin":{"type":"string","description":"The starting location"},"destination":{"type":"string","description":"The destination location"},"mode":{"type":"string","description":"The mode of transportation"}},"required":["origin","destination","mode"]}}},{"type": "function","function":{"name":"generate_password","description":"Generate a random password","parameters":{"type":"object","properties":{"length":{"type":"integer","description":"The length of the password"}},"required":["length"]}}}]

    if not msgs or len(msgs) == 0:
        msgs = [{"role": "system", "content":system_prompt} ,{"role": "user", "content": user_query}]
    else:
        msgs.append({"role": "user", "content": user_query})

    res = chat_method(model="gpt-4-0125-preview", functions=functions, msgs=msgs)
    res_next = res
    if res_next.message.content and len(res_next.message.content) > 0:
        print("\n[AI response]:\n", res_next.message.content)
    else:
        print("\n[AI calling functions]:")
        for tool_call in res_next.message.tool_calls:
            print(f"Tool Call: {tool_call.function}")
    l = 0
    while res_next.message.tool_calls and len(res_next.message.tool_calls) > 0:
        msgs = insert_tool_response(res_next, msgs)

        res_next = chat_method(model="gpt-4-0125-preview", functions=functions, msgs=msgs)
        # for m in msgs:
        #     print(m)
        print(f"Loop {l}")
        if res_next.message.content and len(res_next.message.content) > 0:
            print("\n[AI response]:\n", res_next.message.content)
        else:
            print("\n[AI calling functions]:")
            for tool_call in res_next.message.tool_calls:
                print(f"Tool Call: {tool_call.function}")
        l += 1
    

## Multi + Parallel Function Call

In [4]:
import openai
local_api_key = "sk-"
local_base_url = "http://localhost:8019/v1/"

In [5]:

get_mistral_rubra_response = partial(get_oai_response, api_key=local_api_key, base_url=local_base_url)

user_query = "What is the distance between San Francisco and Cupertino by driving and by air from both directions?"
# user_query = "What is four plus six? What is the result of that plus 2? Take the result and multiply by 5 and then divide by two"
# user_query = "what's the distance between SF and NYC? Use the result value to multiply by 8, and then divide by 2, and then minus 30"
msgs = run_completion(get_mistral_rubra_response, user_query)


Pointing to URL: http://localhost:8019/v1/

[AI calling functions]:
Tool Call: Function(arguments='{"destination":"Cupertino","mode":"driving","origin":"San Francisco"}', name='calculate_distance')
Tool Call: Function(arguments='{"destination":"San Francisco","mode":"driving","origin":"Cupertino"}', name='calculate_distance')
Tool Call: Function(arguments='{"destination":"Cupertino","mode":"air","origin":"San Francisco"}', name='calculate_distance')
Tool Call: Function(arguments='{"destination":"San Francisco","mode":"air","origin":"Cupertino"}', name='calculate_distance')
Pointing to URL: http://localhost:8019/v1/
Loop 0

[AI response]:
  The distance between San Francisco and Cupertino by driving is 50 miles and 100 miles from Cupertino to San Francisco. When traveling by air, the distance is 150 miles from San Francisco to Cupertino and 200 miles from Cupertino to San Francisco.


In [8]:
user_query2 = "now order 3 umbrellas for me and generate a password of length 8"
msgs = run_completion(get_mistral_rubra_response, user_query2, msgs)

Pointing to URL: http://localhost:8019/v1/

[AI calling functions]:
Tool Call: Function(arguments='{"number_to_buy":3}', name='orderUmbrella')
Tool Call: Function(arguments='{"length":8}', name='generate_password')
Pointing to URL: http://localhost:8019/v1/
Loop 0

[AI response]:
  Your order for 3 umbrellas has been placed, and the total price is 10 dollars. The generated password is 96ddefe8.


## Simple Math Chaining

In [9]:
user_query3 = "User tool to help me :  What is four plus six? What is the result of that plus 2? Take the result and multiply by 5 and then divide by two"
msgs = run_completion(get_mistral_rubra_response, user_query3, msgs)

Pointing to URL: http://localhost:8019/v1/

[AI calling functions]:
Tool Call: Function(arguments='{"a":"4","b":"6"}', name='addition')
Tool Call: Function(arguments='{"a":"result","b":"2"}', name='addition')
Tool Call: Function(arguments='{"a":"result","b":"5"}', name='multiplication')
Tool Call: Function(arguments='{"a":"result","b":"2"}', name='division')


ValueError: could not convert string to float: 'result'